In [ ]:
import torch
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 100
EPOCHS = 50

In [ ]:
from sklearn.model_selection import train_test_split
# Import embeddings
features = np.load("/content/drive/MyDrive/Sentiment Analysis/Sentiment Classifier/clean_data/train_embedding.npy")
df = pd.read_csv("/content/drive/MyDrive/Sentiment Analysis/Sentiment Classifier/clean_data/cleaned_train.csv")
df = df[df["text"].notna()].reset_index(drop=True)

labels = np.empty((0, 3))
for index, row in df.iterrows():
   text = row["sentiment"]

   if (text == "neutral"):
      lst = [1,0,0]
   elif (text == "positive"):
      lst = [0,1,0]
   else: lst = [0,0,1]
   
   labels = np.append(labels, np.array(lst, dtype="float64").reshape(1,-1), axis=0)

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2)

# Convert to dataset
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))

train_ds = train_ds.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_ds = test_ds.batch(BATCH_SIZE)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(256,), dtype="float64", name="Input_embeddings"),
    tf.keras.layers.Dense(256, activation="sigmoid", name="Hidden_layer_1"),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(256, activation="sigmoid", name="Hidden_layer_2"),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(3, activation="softmax", name="Classifier")
])


loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=False)


# steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
# num_train_steps = steps_per_epoch * EPOCHS
# num_warmup_steps = int(0.1*num_train_steps)

# init_lr = 3e-5
# optimizer = optimization.create_optimizer(init_lr=init_lr,
#                                           num_train_steps=num_train_steps,
#                                           num_warmup_steps=num_warmup_steps,
#                                           optimizer_type='adamw')

model.compile(optimizer="adam",
              loss=loss_fn,
              metrics=["accuracy"])

In [ ]:
print(torch.nn.functional.softmax(torch.tensor([0.3,-2.9,0.7]), dim=0))


tensor([0.3949, 0.0161, 0.5891])


In [ ]:
model.fit(train_ds, epochs=EPOCHS)

Epoch 1/10
685/685 [==============================] - 2s 3ms/step - loss: 1.0133 - accuracy: 0.4852
Epoch 2/10
685/685 [==============================] - 2s 3ms/step - loss: 0.9508 - accuracy: 0.5392
Epoch 3/10
685/685 [==============================] - 2s 3ms/step - loss: 0.9310 - accuracy: 0.5554
Epoch 4/10
685/685 [==============================] - 2s 3ms/step - loss: 0.9235 - accuracy: 0.5601
Epoch 5/10
685/685 [==============================] - 2s 3ms/step - loss: 0.9117 - accuracy: 0.5666
Epoch 6/10
685/685 [==============================] - 2s 3ms/step - loss: 0.9045 - accuracy: 0.5711
Epoch 7/10
685/685 [==============================] - 2s 3ms/step - loss: 0.9005 - accuracy: 0.5786
Epoch 8/10
685/685 [==============================] - 2s 3ms/step - loss: 0.8945 - accuracy: 0.5759
Epoch 9/10
685/685 [==============================] - 2s 3ms/step - loss: 0.8910 - accuracy: 0.5824
Epoch 10/10
685/685 [==============================] - 2s 3ms/step - loss: 0.8861 - accuracy: 0.5855

In [ ]:
model.evaluate(test_ds, verbose=2)

172/172 - 1s - loss: 0.8947 - accuracy: 0.5935 - 597ms/epoch - 3ms/step


[0.8947167992591858, 0.5935048460960388]

In [ ]:
model.save("/content/drive/MyDrive/Sentiment Analysis/models/neural-network-sentiment-classification")